[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/QuantLet/EMQA/blob/main/EMQA_crack_spread/EMQA_crack_spread.ipynb)

# EMQA_crack_spread
Crack spread: Oil vs Gasoline -- a key refining margin indicator.
**Output:** `crack_spread.pdf`

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')

plt.rcParams.update({
    'figure.facecolor': 'none',
    'axes.facecolor': 'none',
    'savefig.facecolor': 'none',
    'savefig.transparent': True,
    'axes.grid': False,
    'axes.spines.top': False,
    'axes.spines.right': False,
    'font.size': 11,
    'figure.figsize': (12, 6),
})

COLORS = {
    'blue': '#1A3A6E', 'red': '#CD0000', 'green': '#2E7D32',
    'orange': '#E67E22', 'purple': '#8E44AD', 'gray': '#808080',
    'cyan': '#00BCD4', 'amber': '#B5853F'
}

def save_fig(fig, name):
    fig.savefig(name, bbox_inches='tight', transparent=True, dpi=300)
    print(f"Saved: {name}")


In [ ]:
import yfinance as yf

def fetch(ticker, start='2020-01-01', end='2025-12-31'):
    d = yf.download(ticker, start=start, end=end, progress=False)
    if isinstance(d.columns, pd.MultiIndex):
        return d['Close'].squeeze().dropna()
    return d['Close'].dropna()


In [ ]:
# Fetch Brent and RBOB Gasoline
brent = fetch('BZ=F')
gasoline = fetch('RB=F')

common = brent.index.intersection(gasoline.index)
brent = brent.loc[common]
gasoline = gasoline.loc[common]

# Crack spread (simplified 1:1): Gasoline (per gallon * 42 gal/bbl) - Oil (per bbl)
crack = gasoline * 42 - brent
print(f"Crack spread: mean={crack.mean():.2f}, std={crack.std():.2f}")

In [ ]:
fig, axes = plt.subplots(2, 1, figsize=(12, 8), sharex=True)

# Panel 1 - Normalized prices
brent_norm = brent / brent.iloc[0] * 100
gas_norm = gasoline / gasoline.iloc[0] * 100
axes[0].plot(brent_norm.index, brent_norm.values, color=COLORS['blue'], linewidth=1.2, label='Brent Crude')
axes[0].plot(gas_norm.index, gas_norm.values, color=COLORS['orange'], linewidth=1.2, label='RBOB Gasoline')
axes[0].set_ylabel('Normalized Price (base=100)')
axes[0].set_title('Brent Crude vs RBOB Gasoline (Normalized)')
axes[0].legend(loc='upper center', bbox_to_anchor=(0.5, -0.05), frameon=False, ncol=2)

# Panel 2 - Spread with mean +/- std bands
crack_mean = crack.mean()
crack_std = crack.std()
axes[1].plot(crack.index, crack.values, color=COLORS['green'], linewidth=1, label='Crack Spread')
axes[1].axhline(crack_mean, color=COLORS['blue'], linestyle='-', linewidth=1.5, label=f'Mean ({crack_mean:.1f})')
axes[1].axhline(crack_mean + crack_std, color=COLORS['red'], linestyle='--', linewidth=1, alpha=0.7, label=f'+1 Std ({crack_mean + crack_std:.1f})')
axes[1].axhline(crack_mean - crack_std, color=COLORS['red'], linestyle='--', linewidth=1, alpha=0.7, label=f'-1 Std ({crack_mean - crack_std:.1f})')
axes[1].fill_between(crack.index, crack_mean - crack_std, crack_mean + crack_std,
                      alpha=0.1, color=COLORS['red'])
axes[1].set_ylabel('Crack Spread ($/bbl)')
axes[1].set_xlabel('Date')
axes[1].set_title('Crack Spread (Gasoline x 42 - Brent)')
axes[1].legend(loc='upper center', bbox_to_anchor=(0.5, -0.10), frameon=False, ncol=4)

plt.tight_layout()
save_fig(fig, 'crack_spread.pdf')
plt.show()